In [7]:
import cv2
import numpy as np
import os
import random
import math

def zoom_image(image, scale=1.0, is_label=False):
    """Melakukan zoom pada gambar atau label dengan padding atau cropping."""
    h, w = image.shape[:2]
    new_h, new_w = int(h * scale), int(w * scale)
    
    interp = cv2.INTER_NEAREST if is_label else cv2.INTER_LINEAR
    resized = cv2.resize(image, (new_w, new_h), interpolation=interp)

    if scale > 1.0:
        start_h, start_w = (new_h - h) // 2, (new_w - w) // 2
        return resized[start_h:start_h + h, start_w:start_w + w]
    else:
        pad_h = (h - new_h) / 2
        pad_w = (w - new_w) / 2
        top, bottom = math.floor(pad_h), math.ceil(pad_h)
        left, right = math.floor(pad_w), math.ceil(pad_w)
        
        return cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)

def data_augment(source_folder_images, source_folder_labels, source_folder_labels1, destination_folder_images, destination_folder_labels, destination_folder_labels1, augmentations_per_image=5):
    """Melakukan augmentasi data dengan flipping, rotasi, dan zooming."""
    
    os.makedirs(destination_folder_images, exist_ok=True)
    os.makedirs(destination_folder_labels, exist_ok=True)
    os.makedirs(destination_folder_labels1, exist_ok=True)
    
    listing_images = sorted(os.listdir(source_folder_images))

    for file_name in listing_images:
        if file_name.endswith('.png'):
            image_path = os.path.join(source_folder_images, file_name)
            label_path = os.path.join(source_folder_labels, file_name)
            label_path1 = os.path.join(source_folder_labels1, file_name)
            
            if not os.path.exists(label_path) or not os.path.exists(label_path1):
                print(f"⚠ Label untuk {file_name} tidak ditemukan di salah satu folder, melewati file ini.")
                continue

            # Membaca gambar dan label
            image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
            label = cv2.imread(label_path, cv2.IMREAD_UNCHANGED)
            label1 = cv2.imread(label_path1, cv2.IMREAD_UNCHANGED)

            for i in range(augmentations_per_image):
                # Flip Vertikal (Sinkron)
                flipped_v_img = cv2.flip(image, 0)
                flipped_v_lbl = cv2.flip(label, 0)
                flipped_v_lbl1 = cv2.flip(label1, 0)
                cv2.imwrite(os.path.join(destination_folder_images, f"fv_{i}_{file_name}"), flipped_v_img)
                cv2.imwrite(os.path.join(destination_folder_labels, f"fv_{i}_{file_name}"), flipped_v_lbl)
                cv2.imwrite(os.path.join(destination_folder_labels1, f"fv_{i}_{file_name}"), flipped_v_lbl1)  # BENAR

                # Flip Horizontal (Sinkron)
                flipped_h_img = cv2.flip(image, 1)
                flipped_h_lbl = cv2.flip(label, 1)
                flipped_h_lbl1 = cv2.flip(label1, 1)
                cv2.imwrite(os.path.join(destination_folder_images, f"fh_{i}_{file_name}"), flipped_h_img)
                cv2.imwrite(os.path.join(destination_folder_labels, f"fh_{i}_{file_name}"), flipped_h_lbl)
                cv2.imwrite(os.path.join(destination_folder_labels1, f"fh_{i}_{file_name}"), flipped_h_lbl1)  # BENAR

                # Rotasi dengan sudut random (Sinkron)
                random_angle = random.uniform(0, 360)
                h, w = image.shape[:2]
                M = cv2.getRotationMatrix2D((w / 2, h / 2), random_angle, 1)

                rotated_img = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
                rotated_lbl = cv2.warpAffine(label, M, (w, h), flags=cv2.INTER_NEAREST, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
                rotated_lbl1 = cv2.warpAffine(label1, M, (w, h), flags=cv2.INTER_NEAREST, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
                
                cv2.imwrite(os.path.join(destination_folder_images, f"rot_{i}_{int(random_angle)}_{file_name}"), rotated_img)
                cv2.imwrite(os.path.join(destination_folder_labels, f"rot_{i}_{int(random_angle)}_{file_name}"), rotated_lbl)
                cv2.imwrite(os.path.join(destination_folder_labels1, f"rot_{i}_{int(random_angle)}_{file_name}"), rotated_lbl1)  # BENAR

                # Zoom (Sinkron)
                zoom_factor = random.uniform(0.8, 1.2)
                zoomed_img = zoom_image(image, zoom_factor, is_label=False)
                zoomed_lbl = zoom_image(label, zoom_factor, is_label=True)
                zoomed_lbl1 = zoom_image(label1, zoom_factor, is_label=True)
                cv2.imwrite(os.path.join(destination_folder_images, f"zoom_{i}_{zoom_factor:.2f}_{file_name}"), zoomed_img)
                cv2.imwrite(os.path.join(destination_folder_labels, f"zoom_{i}_{zoom_factor:.2f}_{file_name}"), zoomed_lbl)
                cv2.imwrite(os.path.join(destination_folder_labels1, f"zoom_{i}_{zoom_factor:.2f}_{file_name}"), zoomed_lbl1)  # BENAR

# Path dataset
source_folder_images = r"D:\1_dina\BINER\images"
source_folder_labels = r"D:\1_dina\BINER\vessel-segmentations"
source_folder_labels1 = r"D:\Gio&Maul\maul\LES-AV\arteries-and-veins"
destination_folder_images = r"D:\1_dina\BINER\IMG"
destination_folder_labels = r"D:\1_dina\BINER\LABEL"
destination_folder_labels1 = r"D:\1_dina\LES AV_JURNAL\LABEL"

# Jalankan augmentasi dengan 10 variasi per gambar
data_augment(source_folder_images, source_folder_labels, source_folder_labels1, destination_folder_images, destination_folder_labels, destination_folder_labels1, augmentations_per_image=10)

In [8]:
import os
import cv2
import numpy as np
import random

def apply_gamma_bilateral_to_folder(input_folder, output_folder, d):
    # Membuat folder output jika belum ada
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Mendapatkan daftar file dalam folder input
    image_files = os.listdir(input_folder)
    
    for image_file in image_files:
        # Membuat jalur lengkap untuk citra input
        input_image_path = os.path.join(input_folder, image_file)
        
        # Membaca citra
        img = cv2.imread(input_image_path)
        
        # Random nilai Gamma Correction
        gamma = random.uniform(0.5, 2.5)
        inv_gamma = 1.0 / gamma
        table = np.array([(i / 255.0) ** inv_gamma * 255 for i in np.arange(0, 256)]).astype("uint8")
        img_gamma = cv2.LUT(img, table)
        
        # Random nilai Sigma Color dan Sigma Space untuk Bilateral Filtering
        sigma_color = random.randint(50, 150)
        sigma_space = random.randint(50, 150)
        img_bilateral = cv2.bilateralFilter(img_gamma, d, sigma_color, sigma_space)
        
        # Membuat jalur lengkap untuk citra output
        output_image_path = os.path.join(output_folder, image_file)
        
        # Menyimpan citra hasil ke folder output
        cv2.imwrite(output_image_path, img_bilateral)
        
        # Menampilkan nilai random yang digunakan
        print(f"File: {image_file} | Gamma: {gamma:.2f} | Sigma Color: {sigma_color} | Sigma Space: {sigma_space}")

    print("\nGamma Correction dan Bilateral Filtering telah diterapkan pada semua citra dalam folder.")

# Contoh penggunaan
input_folder = "D:/1_dina/BINER/IMG"
output_folder = "D:/1_dina/BINER/PROCESSED"
bilateral_d = 9  # Diameter filter bilateral

apply_gamma_bilateral_to_folder(input_folder, output_folder, bilateral_d)

File: fh_0_111.png | Gamma: 2.40 | Sigma Color: 101 | Sigma Space: 104
File: fh_0_119.png | Gamma: 1.91 | Sigma Color: 119 | Sigma Space: 68
File: fh_0_12.png | Gamma: 2.05 | Sigma Color: 66 | Sigma Space: 143
File: fh_0_15.png | Gamma: 2.26 | Sigma Color: 63 | Sigma Space: 84
File: fh_0_224.png | Gamma: 1.36 | Sigma Color: 70 | Sigma Space: 79
File: fh_0_240.png | Gamma: 1.54 | Sigma Color: 104 | Sigma Space: 142
File: fh_0_275.png | Gamma: 0.73 | Sigma Color: 123 | Sigma Space: 108
File: fh_0_279.png | Gamma: 2.30 | Sigma Color: 150 | Sigma Space: 145
File: fh_0_304.png | Gamma: 0.85 | Sigma Color: 134 | Sigma Space: 108
File: fh_0_318.png | Gamma: 1.68 | Sigma Color: 75 | Sigma Space: 96
File: fh_0_33.png | Gamma: 2.37 | Sigma Color: 99 | Sigma Space: 139
File: fh_0_34.png | Gamma: 2.45 | Sigma Color: 75 | Sigma Space: 93
File: fh_0_367.png | Gamma: 2.32 | Sigma Color: 69 | Sigma Space: 84
File: fh_0_37.png | Gamma: 0.90 | Sigma Color: 108 | Sigma Space: 116
File: fh_0_370.png | Gam

In [9]:
import cv2
import numpy as np
import os
import random
from skimage.metrics import structural_similarity as ssim

def calculate_mse(original, processed):
    return np.mean((original - processed) ** 2)

def calculate_rmse(original, processed):
    return np.sqrt(calculate_mse(original, processed))

def calculate_psnr(original, processed):
    mse = calculate_mse(original, processed)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    return 20 * np.log10(max_pixel / np.sqrt(mse))

def calculate_ssim(original, processed):
    original_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY) if len(original.shape) == 3 else original
    processed_gray = cv2.cvtColor(processed, cv2.COLOR_BGR2GRAY) if len(processed.shape) == 3 else processed
    score, _ = ssim(original_gray, processed_gray, full=True)
    return score

def find_matching_file(base_name, processed_files):
    candidates = [f for f in processed_files if base_name in os.path.splitext(f)[0]]
    return candidates[0] if candidates else None

def evaluate_images(source_folder_images, processed_folder_images, sample_size=10):
    if not os.path.exists(source_folder_images) or not os.path.exists(processed_folder_images):
        print("Error: One or both of the specified directories do not exist.")
        return
    
    listing_images = sorted([f for f in os.listdir(source_folder_images) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.tif'))])
    processed_files = set(os.listdir(processed_folder_images))  # Semua file hasil

    if not listing_images:
        print("Error: No images found in the source folder.")
        return
    
    sampled_images = random.sample(listing_images, min(sample_size, len(listing_images)))

    for file_name in sampled_images:
        base_name = os.path.splitext(file_name)[0]
        matched_file = find_matching_file(base_name, processed_files)

        if not matched_file:
            print(f"Processed image for {file_name} not found in {processed_folder_images}. Skipping.")
            continue

        original_path = os.path.join(source_folder_images, file_name)
        processed_path = os.path.join(processed_folder_images, matched_file)

        original = cv2.imread(original_path, cv2.IMREAD_UNCHANGED)
        processed = cv2.imread(processed_path, cv2.IMREAD_UNCHANGED)

        if original is None or processed is None:
            print(f"Error: Could not read {file_name} or its processed version {matched_file}. Skipping.")
            continue

        mse_value = calculate_mse(original, processed)
        rmse_value = calculate_rmse(original, processed)
        psnr_value = calculate_psnr(original, processed)
        ssim_value = calculate_ssim(original, processed)

        print(f"Image: {file_name} -> Matched: {matched_file}")
        print(f"MSE: {mse_value:.2f}")
        print(f"RMSE: {rmse_value:.2f}")
        print(f"PSNR: {psnr_value:.2f} dB")
        print(f"SSIM: {ssim_value:.4f}")
        print("------------------------")

# Folder sumber dan tujuan
source_folder_images = "D:/1_dina/BINER/IMG"
processed_folder_images = "D:/1_dina/BINER/PROCESSED"

evaluate_images(source_folder_images, processed_folder_images, sample_size=10)

Image: fh_6_34.png -> Matched: fh_6_34.png
MSE: 84.82
RMSE: 9.21
PSNR: 28.85 dB
SSIM: 0.6759
------------------------
Image: rot_6_49_275.png -> Matched: rot_6_49_275.png
MSE: 3.07
RMSE: 1.75
PSNR: 43.26 dB
SSIM: 0.9814
------------------------
Image: fh_8_33.png -> Matched: fh_8_33.png
MSE: 9.91
RMSE: 3.15
PSNR: 38.17 dB
SSIM: 0.9757
------------------------
Image: fh_8_37.png -> Matched: fh_8_37.png
MSE: 81.84
RMSE: 9.05
PSNR: 29.00 dB
SSIM: 0.9356
------------------------
Image: fh_1_12.png -> Matched: fh_1_12.png
MSE: 81.29
RMSE: 9.02
PSNR: 29.03 dB
SSIM: 0.8049
------------------------
Image: fh_4_119.png -> Matched: fh_4_119.png
MSE: 75.10
RMSE: 8.67
PSNR: 29.37 dB
SSIM: 0.9268
------------------------
Image: fv_0_53.png -> Matched: fv_0_53.png
MSE: 81.16
RMSE: 9.01
PSNR: 29.04 dB
SSIM: 0.9062
------------------------
Image: fh_4_240.png -> Matched: fh_4_240.png
MSE: 58.17
RMSE: 7.63
PSNR: 30.48 dB
SSIM: 0.7944
------------------------
Image: rot_1_53_279.png -> Matched: rot_1_53